# Note
This code produce the dataframe with the lat, lon of buildings, walking distance and time matrix.

Should only be run once and the output is saved to reduce call to Google api.

In [1]:
import pandas as pd
import json
import requests
import math
import numpy as np
from haversine import haversine

with open("config.json") as f:
    settings = json.load(f)

# Getting the Geolocation

In [2]:
# Reading in the release data
release_pts = pd.read_excel("Yishun Wolbachia Release Points.xlsx")
release_pts["NumRelease"] = release_pts["ReleaseLocation"].str.split(",").apply(len)
release_pts["Block"] = release_pts["Block"].astype("str")
release_pts["Block"] = release_pts["Block"].str.replace(".Bin Centre", "")
display(release_pts)

,Location,Postal Code,Block,ReleaseLocation,NumRelease
0,YISHUN STREET 21,760201.0,201,"51,53",2
1,YISHUN STREET 21,760201.0,201,"55,57",2
2,YISHUN STREET 21,760201.0,201,"59,61",2
3,YISHUN STREET 21,760201.0,201,"63,65",2
4,YISHUN STREET 21,760201.0,201,"67,69",2
...,...,...,...,...,...
395,YISHUN RING ROAD,760312.0,312,"1216,1218",2
396,YISHUN RING ROAD,760314.0,314,"1180,1182",2
397,YISHUN RING ROAD,760314.0,314,"1184,1186",2
398,YISHUN RING ROAD,760314.0,314,"1188,1190",2


Adding the number of release points to the df.

In [3]:
def count_release(df_input):
    """ Getting the number of release points at each block
    """
    output = {"Road": df_input.iloc[0, 0], "NumRelease": df_input["NumRelease"].sum()}
    return pd.Series(output)

In [4]:
df = release_pts.groupby("Block").apply(count_release).reset_index()
df["address"] = df["Block"] + "+" + df["Road"].str.replace(" ", "+")
df["location"] = None
display(df)

,Block,Road,NumRelease,address,location
0,201,YISHUN STREET 21,10,201+YISHUN+STREET+21,None
1,202,YISHUN STREET 21,10,202+YISHUN+STREET+21,None
2,203,YISHUN STREET 21,10,203+YISHUN+STREET+21,None
3,204,YISHUN STREET 21,10,204+YISHUN+STREET+21,None
4,205,YISHUN STREET 21,10,205+YISHUN+STREET+21,None
...,...,...,...,...,...
79,309,YISHUN RING ROAD,11,309+YISHUN+RING+ROAD,None
80,310,YISHUN RING ROAD,6,310+YISHUN+RING+ROAD,None
81,311,YISHUN RING ROAD,12,311+YISHUN+RING+ROAD,None
82,312,YISHUN RING ROAD,12,312+YISHUN+RING+ROAD,None


In [ ]:
url = "https://maps.googleapis.com/maps/api/geocode/json"

for each in range(num_blonum_blocks):
    if not df.loc[each, "location"]:
        params = dict(address=df.loc[each, "address"], key=settings["api_key"])
        resp = requests.get(url=url, params=params)
        data = resp.json()
        try:
            df.loc[each, "location"] = (
                str(data["results"][0]["geometry"]["location"]["lat"])
                + ","
                + str(data["results"][0]["geometry"]["location"]["lng"])
            )
        except:
            print(each)
            print(data)

df.to_pickle("data/block_with_geoloc.pickle")

# Getting the walking distance and time 

In [6]:
# splitting the addresses into multiple string due to api limit

arr = ""
arr_total = []
count = 0
for index, row in df.iterrows():
    count += 1
    arr += str(row["address"]) + "|"
    if count >= 25:
        arr_total.append(arr)
        count = 0
        arr = ""
if count != 0:
    arr_total.append(arr)
arr_total

['201+YISHUN+STREET+21|202+YISHUN+STREET+21|203+YISHUN+STREET+21|204+YISHUN+STREET+21|205+YISHUN+STREET+21|206+YISHUN+STREET+21|207+YISHUN+STREET+21|208+YISHUN+STREET+21|209+YISHUN+STREET+21|210+YISHUN+STREET+21|211+YISHUN+STREET+21|212+YISHUN+STREET+21|213+YISHUN+STREET+21|214+YISHUN+STREET+21|215+YISHUN+STREET+21|216+YISHUN+STREET+21|217+YISHUN+STREET+21|218+YISHUN+STREET+21|219+YISHUN+STREET+21|220+YISHUN+STREET+21|221+YISHUN+STREET+21|222+YISHUN+STREET+21|223+YISHUN+STREET+21|224+YISHUN+STREET+21|225+YISHUN+STREET+21|',
 '226+YISHUN+STREET+21|227+YISHUN+STREET+21|228+YISHUN+STREET+21|229+YISHUN+STREET+21|230+YISHUN+STREET+21|231+YISHUN+STREET+21|232+YISHUN+STREET+21|233+YISHUN+STREET+21|234+YISHUN+STREET+21|235+YISHUN+STREET+21|236+YISHUN+RING+ROAD|237+YISHUN+RING+ROAD|238+YISHUN+RING+ROAD|239+YISHUN+RING+ROAD|240+YISHUN+RING+ROAD|241+YISHUN+RING+ROAD|242+YISHUN+RING+ROAD|243+YISHUN+RING+ROAD|244+YISHUN+RING+ROAD|245+YISHUN+AVENUE+9|246+YISHUN+AVENUE+9|247+YISHUN+AVENUE+9|248+YISHU

In [ ]:
# WILL RUN FOR 2 MINS AT MAX

num_blocks = len(df)
time_matrix = np.zeros((num_blocks, num_blocks))
time_array = []
dist_matrix = np.zeros((num_blocks, num_blocks))
dist_array = []

url_google_route = "https://maps.googleapis.com/maps/api/distancematrix/json"


for each_index in range(len(df)):
    time_array = []
    dist_array = []
    print("Inititaing " + str(each_index) + "/83 first...")
    for index, dest_input in enumerate(arr_total):
        params = dict(
            destinations=dest_input,
            origins=df.iloc[each_index]["address"],
            key=settings["api_key"],
            mode="walking",
        )
        resp = requests.get(url=url_google_route, params=params)
        data = resp.json()
        for each in range(len(data["rows"][0]["elements"])):
            if data["rows"][0]["elements"][each]["status"] == "OK":
                time_array.append(
                    data["rows"][0]["elements"][each]["duration"]["value"]
                )
                dist_array.append(
                    data["rows"][0]["elements"][each]["distance"]["value"]
                )
            else:
                print(each)
                print("Origin: {}, Destination: ".format(data["origin_addresses"][0]))
                print(data)
    print(each_index)
    time_matrix[each_index] = time_array
    dist_matrix[each_index] = dist_array

In [ ]:
np.save("data/time_matrix.npy", time_matrix)
np.save("data/dist_matrix.npy", dist_matrix)